In [1]:
import pandas as pd
from tqdm import tqdm
import json
import time
import openai
import re
from transformers import GPT2Tokenizer

In [2]:
df = pd.read_csv('sample.csv')
df.head()

,id,author_id,in_reply_to_user_id,quoted_user_id,retweeted_user_id,text,public_metrics.like_count,public_metrics.quote_count,public_metrics.reply_count,public_metrics.retweet_count,...,url,link_text,party,name,Occupation(s),Education,Residence[5],sentiment,age,gender
0,1174499599181959168,262192574,NaN,NaN,NaN,We should never forget the sacrifices made by ...,22,0,0,2,...,https://www.wdam.com/2019/09/18/senator-cindy-...,MISSISSIPPI (WLOX) - U.S. Senator Cindy Hyde-S...,Republican,Cindy Hyde-Smith,Lobbyist Farmer,Copiah-Lincoln Community College (AA) Universi...,Brookhaven,Positive,65,female
1,1320738770543628288,15808765,NaN,NaN,NaN,It's no secret what Donald Trump and Senate Re...,1494,67,264,323,...,https://thehill.com/homenews/administration/52...,President Trump says in an interview with “60 ...,Democrat,Cory Booker,Lawyer,"Stanford University (BA, MA) Queen's College, ...",Newark,Negative,55,male
2,998614598424068101,15745368,NaN,NaN,NaN,Some #Florida elections officials didn’t like ...,259,19,105,112,...,https://www.washingtonpost.com/news/powerpost/...,Comment on this story Comment Gift Article Sha...,Republican,Marco Rubio,Lawyer,University of Florida (BA) University of Miami...,West Miami[13],Neutral,53,male
3,955084959840129024,13218102,NaN,NaN,NaN,"One woman, two marches: Can women who oppose a...",14,7,61,9,...,https://www.dallasnews.com/opinion/commentary/...,Rachel Lamb saw as much fate as coincidence in...,Republican,John Cornyn,Lawyer,"Trinity University (BA) St. Mary's University,...",Austin,Neutral,72,male
4,1255486188892422149,1080960924687704064,NaN,NaN,NaN,The coronavirus pandemic continues to highligh...,8,1,1,2,...,https://www.kfvs12.com/2020/04/29/sen-hawley-c...,WASHINGTON D.C. (KFVS) - U.S. Senators Josh Ha...,Republican,Josh Hawley,Lawyer Professor,Stanford University (BA) Yale University (JD),Ozark,Neutral,44,male


In [3]:
def make_prompt(party, age, news_content): 
    prompt = (
        f"You are a politician arguing on Twitter and a representative of the {party} party. You are {age} years old."
        f"Read the article below and write a tweet with no more than 280 characters to comment on it.\n\n"
        f"Article:\n{news_content}"
    )
    return prompt

In [4]:
# Function to truncate text to fit within a token limit

# initialize the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

def truncate_prompt(prompt, max_tokens, tokenizer):
    tokens = tokenizer.encode(prompt)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
    return tokenizer.decode(tokens)

In [6]:
openai.api_base = "https://api.openai.com/v1/"
#openai.api_key = "xxx"

In [7]:
# save progress 
all_responses = []
current_row_index = 0 

In [8]:
# generate AI responses

while current_row_index < df.shape[0]:
    try:
        for ind, row in tqdm(df[current_row_index:].iterrows(), total=df[current_row_index:].shape[0]):
            prompt = make_prompt(row["party"], row["age"], row['link_text'])
            
            # check token length and truncate if necessary
            if len(tokenizer.encode(prompt)) > 16385:  #max input tokens for the model
                prompt = truncate_prompt(prompt, 16385, tokenizer)
            
            responses = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=280,
                n=1
            )
            response_list = [row['id'], row['text']]
            response_list.extend([i['message']['content'] for i in responses['choices']])
            all_responses.append(response_list)
            current_row_index = ind + 1  # move to the next row

    except KeyboardInterrupt:
        print(f"Interrupted at row {current_row_index}. Saving progress...")
    
    except Exception as e:
        print(f"An error occurred at row {current_row_index}. Error: {str(e)}. Retrying...")

        print("Completed or interrupted. Final progress saved.")

  1%|▌                                    | 140/10000 [03:14<3:48:39,  1.39s/it]


An error occurred at row 140. Error: Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_446c360a8e44e941c6d36c76ddd49600 in your message.) {
  "error": {
    "message": "Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_446c360a8e44e941c6d36c76ddd49600 in your message.)",
    "type": null,
    "param": null,
    "code": null
  }
} 500 {'error': {'message': 'Failed to create completion as the model generated 

 44%|███████████████▎                   | 4299/9860 [1:41:44<2:11:36,  1.42s/it]


An error occurred at row 4439. Error: Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_4afa586a08f59d989a58938ad3162765 in your message.) {
  "error": {
    "message": "Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_4afa586a08f59d989a58938ad3162765 in your message.)",
    "type": null,
    "param": null,
    "code": null
  }
} 500 {'error': {'message': 'Failed to create completion as the model generated

 13%|████▊                                 | 709/5561 [16:44<1:54:36,  1.42s/it]


An error occurred at row 5148. Error: Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_60a54731e3663932066ea1ded25cad73 in your message.) {
  "error": {
    "message": "Failed to create completion as the model generated invalid Unicode output. Unfortunately, this can happen in rare situations. Consider reviewing your prompt or reducing the temperature of your request. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID req_60a54731e3663932066ea1ded25cad73 in your message.)",
    "type": null,
    "param": null,
    "code": null
  }
} 500 {'error': {'message': 'Failed to create completion as the model generated

100%|█████████████████████████████████████| 4852/4852 [1:55:27<00:00,  1.43s/it]


In [9]:
len(all_responses)

10000

In [10]:
#save progress
pd.DataFrame(all_responses, columns = ['id', 'text', 'tweet_gpt_age']).to_csv('tweet_gpt_age.csv', index=False)

In [11]:
df2 = pd.read_csv('tweet_gpt_age.csv')
df2.head()

,id,text,tweet_gpt_age
0,1174499599181959168,We should never forget the sacrifices made by ...,"As a proud Republican, I fully support U.S. Se..."
1,1320738770543628288,It's no secret what Donald Trump and Senate Re...,"As a Democrat, I am deeply troubled by Trump's..."
2,998614598424068101,Some #Florida elections officials didn’t like ...,The security of our election systems is a cruc...
3,955084959840129024,"One woman, two marches: Can women who oppose a...",Rachel Lamb exemplifies the importance of find...
4,1255486188892422149,The coronavirus pandemic continues to highligh...,"As a Republican leader, I support efforts to u..."


## Labelling

In [12]:
#function to create prompt
def create_prompt(tweet):
    text = 'Classify the sentiment of the following tweet as "Positive", "Negative" or "Neutral" like in the examples: Tweet: "I’m not surprised Alaska ranks at the top for new Army recruits’ preferred duty station. And once they arrive here, it’s our job to make sure they have the support they need both on and off duty" Sentiment: Positive, Tweet: "This would be deeply ironic if it wasn’t so absolutely infuriating" Sentiment: Negative, Tweet: "Given the U.S. is on the brink of recession and much of the world is facing catastrophically high energy prices, the completion of the regulatory process for the Alaska LNG Project must be a top priority for @ENERGY" Sentiment: Neutral, Tweet: '
    tweet = tweet 
    prompt = text + tweet + ' Sentiment:'
    return prompt

In [14]:
# save progress 
all_responses = []
current_row_index = 0 

In [38]:
# generate sentiment labels for each tweet 
try:
    while current_row_index < df2.shape[0]:
        for ind, row in tqdm(df2[current_row_index:].iterrows(), total=df2[current_row_index:].shape[0]):
            prompt = create_prompt(row['tweet_gpt_age'])
            responses = openai.ChatCompletion.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=2,
                n=1
            )
            response_list = [row['id'], row['text'], row['tweet_gpt_age']]
            response_list.extend([i['message']['content'] for i in responses['choices']])
            all_responses.append(response_list)
            current_row_index = ind + 1  # move to the next row

except KeyboardInterrupt:
    print(f"Interrupted at row {current_row_index}. Saving progress...")
    
except Exception as e:
    print(f"An error occurred at row {current_row_index}. Error: {str(e)}. Retrying...")

print("Completed or interrupted. Final progress saved.")

100%|███████████████████████████████████████████| 97/97 [00:40<00:00,  2.42it/s]

Completed or interrupted. Final progress saved.


In [39]:
len(all_responses)

10000

In [40]:
#save progress
pd.DataFrame(all_responses, columns = ['id', 'text', 'tweet_gpt_age', 'label']).to_csv('tweet_gpt_age.csv', index=False)

In [41]:
df2 = pd.read_csv('tweet_gpt_age.csv')
df2.head()

,id,text,tweet_gpt_age,label
0,1174499599181959168,We should never forget the sacrifices made by ...,"As a proud Republican, I fully support U.S. Se...",Positive
1,1320738770543628288,It's no secret what Donald Trump and Senate Re...,"As a Democrat, I am deeply troubled by Trump's...",Negative
2,998614598424068101,Some #Florida elections officials didn’t like ...,The security of our election systems is a cruc...,Positive
3,955084959840129024,"One woman, two marches: Can women who oppose a...",Rachel Lamb exemplifies the importance of find...,Positive
4,1255486188892422149,The coronavirus pandemic continues to highligh...,"As a Republican leader, I support efforts to u...",Positive
